In [3]:
import os
from typing import TypeVar
import torchvision
from torchvision.transforms import v2 as transforms
import torch
import torch.nn as nn
from models.centernet import ModelBuilder
from data.dataset_visualizer import ImageDatasetWithLabels
from torch.utils.data import Subset
from postprocess_visual.object_detection_visualizer import (
    ObjectDetectionVisualizer,
)

def prepare_dataset():
    dataset = torchvision.datasets.VOCDetection(
        root="../VOC", year="2007", image_set="val", download=True
    )
    dataset = torchvision.datasets.wrap_dataset_for_transforms_v2(dataset)
    indices = range(10)
    dataset = Subset(dataset, indices)

    transform = transforms.Compose(
        [
            transforms.Resize(size=(256, 256)),
        ]
    )

    return ImageDatasetWithLabels(dataset=dataset, transformation=transform)

dataset = prepare_dataset()

ModuleType = TypeVar("ModuleType", bound=nn.Module)


def load_model(model_type: ModuleType, checkpoint_path: str = None, alpha=1.0):
    checkpoint_path = (
        "../models/checkpoints/pretrained_weights.pt"
        if checkpoint_path is None
        else checkpoint_path
    )
    if not os.path.exists(checkpoint_path):
        raise FileNotFoundError(f"Checkpoint file not found: {checkpoint_path}")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model_type(alpha=alpha).to(device)
    model.load_state_dict(
        torch.load(
            checkpoint_path,
            map_location=device,
            weights_only=True,
        )
    )
    model.eval()
    return model

model = load_model(ModelBuilder, alpha=1.0, checkpoint_path="../callbacks/imagenet_checkpoints/best_model_loss_1.4433.pt")

custom_transforms = transforms.Compose(
    [
        transforms.ToImage(),
        transforms.ToDtype(torch.float32, scale=True),
    ]
)
preds = []
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for i, orig_img in enumerate(dataset):
    img = custom_transforms(orig_img)
    img = img.unsqueeze(0).to(device)

    with torch.no_grad():
        pred = model(img)
        preds.append(pred)

# Create visualizer with default settings
visualizer = ObjectDetectionVisualizer(
    dataset=dataset, input_height=256, input_width=256, confidence_threshold=0.3
)

# Visualize predictions
visualizer.visualize_predictions(preds)

Using downloaded and verified file: ../VOC\VOCtrainval_06-Nov-2007.tar
Extracting ../VOC\VOCtrainval_06-Nov-2007.tar to ../VOC
loss_dict = {}


RuntimeError: Error(s) in loading state_dict for ModelBuilder:
	Missing key(s) in state_dict: "backbone.layer1.conv_1.weight", "backbone.layer1.conv_1.bias", "backbone.layer1.bn_1.weight", "backbone.layer1.bn_1.bias", "backbone.layer1.bn_1.running_mean", "backbone.layer1.bn_1.running_var", "backbone.layer2.conv_2.weight", "backbone.layer2.conv_2.bias", "backbone.layer2.bn_2.weight", "backbone.layer2.bn_2.bias", "backbone.layer2.bn_2.running_mean", "backbone.layer2.bn_2.running_var", "backbone.layer3.conv_3.weight", "backbone.layer3.conv_3.bias", "backbone.layer3.bn_3.weight", "backbone.layer3.bn_3.bias", "backbone.layer3.bn_3.running_mean", "backbone.layer3.bn_3.running_var", "backbone.layer4.conv_4.weight", "backbone.layer4.conv_4.bias", "backbone.layer4.bn_4.weight", "backbone.layer4.bn_4.bias", "backbone.layer4.bn_4.running_mean", "backbone.layer4.bn_4.running_var", "backbone.layer5.conv_5.weight", "backbone.layer5.conv_5.bias", "backbone.layer5.bn_5.weight", "backbone.layer5.bn_5.b